Mithilfe dieses Notebooks wird die Quelldatei mit Bibtex-Code als Dataframe eingelesen und gemäß den Vorgaben den ACM-Stils in einen Literaturstring konvertiert. Die Ausgabe ist eine csv-Datei mit den vier Spalten Literaturstring | Stil | Literaturtyp | Bibtex.

In [11]:
import pandas as pd

# Pfad zur .bib-Datei mit article-, proceedings- und inproceedings-Einträgen
bib_file = 'anthology.bib'

# Liste zum Speichern der Bibtex-Einträge
bib_entries = []

# Bibtex-Einträge extrahieren
with open(bib_file, 'r', encoding='utf-8') as file:
    entry = ''
    for line in file:
        # Einträge erkennen
        if line.strip().startswith('@'):
            if entry:
                bib_entries.append(entry)
            entry = line
        else:
            entry += line.rstrip() + '\n'
    # Letzten Eintrag hinzufügen
    if entry:
        bib_entries.append(entry)

# DataFrame erstellen
df = pd.DataFrame({'BibTeX': bib_entries})


# Pfad zur .bib-Datei mit incollection-Einträgen
bib_file2 = '1016incollection_reviewed.bib'

# Liste zum Speichern der Bibtex-Einträge
bib_entries2 = []

# Bibtex-Einträge extrahieren
with open(bib_file2, 'r', encoding='utf-8') as file:
    entry = ''
    for line in file:
        # Einträge erkennen
        if line.strip().startswith('@'):
            if entry:
                bib_entries2.append(entry)
            entry = line
        else:
            entry += line.rstrip() + '\n'
    # Letzten Eintrag hinzufügen
    if entry:
        bib_entries2.append(entry)

# DataFrame erstellen
df5 = pd.DataFrame({'BibTeX': bib_entries2})


# Pfad zur .bib-Datei mit book-Einträgen
bib_file3 = '1000books.bib'

# Liste zum Speichern der Bibtex-Einträge
bib_entries3 = []

# Bibtex-Einträge extrahieren
with open(bib_file3, 'r', encoding='utf-8') as file:
    entry = ''
    for line in file:
        # Einträge erkennen
        if line.strip().startswith('@'):
            if entry:
                bib_entries3.append(entry)
            entry = line
        else:
            entry += line.rstrip() + '\n'
    # Letzten Eintrag hinzufügen
    if entry:
        bib_entries3.append(entry)

# DataFrame erstellen
df6 = pd.DataFrame({'BibTeX': bib_entries3})


# Pfad zur .bib-Datei mit book-Einträgen
bib_file4 = '1000PhD.bib'

# Liste zum Speichern der Bibtex-Einträge
bib_entries4 = []

# Bibtex-Einträge extrahieren
with open(bib_file4, 'r', encoding='utf-8') as file:
    entry = ''
    for line in file:
        # Einträge erkennen
        if line.strip().startswith('@'):
            if entry:
                bib_entries4.append(entry)
            entry = line
        else:
            entry += line.rstrip() + '\n'
    # Letzten Eintrag hinzufügen
    if entry:
        bib_entries4.append(entry)

# DataFrame erstellen
df7 = pd.DataFrame({'BibTeX': bib_entries4})

# DataFrame anzeigen
# display(df)

In [12]:
# Kürzung zu Testzwecken
#df = df.head(10000)

In [13]:
# Anpassung der Bibliothek Pybtex für den ACM-Style

import re
import io
from pybtex.database import parse_string
from pybtex.plugin import find_plugin
from pybtex.style.formatting import BaseStyle, toplevel
from pybtex.style.template import (
    field, first_of, href, join, optional, optional_field, sentence, tag,
    together, words, node, FieldIsMissing
)
from pybtex import richtext
from pybtex.richtext import Text, Symbol

firstlast = find_plugin('pybtex.style.names', 'firstlast')()

def format_pages(text):
    dash_re = re.compile(r'-+')
    pages = Text(Symbol('ndash')).join(text.split(dash_re))
    if re.search('[-‒–—―]', str(text)):
        return Text("pp.", Symbol('nbsp'), pages)
    return Text("p.", Symbol('nbsp'), pages)

pages = field('pages', apply_func=format_pages)
date = words[field('year'), optional[", ", field('month')]]

@node
def acm_names(children, context, role, **kwargs):
    """
    Returns formatted names as an APA compliant reference list citation.
    """
    assert not children

    try:
        persons = context['entry'].persons[role]
    except KeyError:
        raise FieldIsMissing(role, context['entry'])

    style = context['style']

    if len(persons) > 20:
        formatted_names = [style.format_name(person) for person in persons[:20]]
        formatted_names += [richtext.Text("et al.")]
        return join(sep=', ')[formatted_names].format_data(context)
    else:
        formatted_names = [style.format_name(person) for person in persons]
        return join(sep=', ', sep2=' and ', last_sep=', and ')[
            formatted_names].format_data(context)

@node
def editor_names(children, context, with_suffix=True, **kwargs):
    """
    Returns formatted editor names for inbook.
    """
    assert not children

    try:
        editors = context['entry'].persons['editor']
    except KeyError:
        raise FieldIsMissing('editor', context['entry'])

    formatted_names = [
        firstlast.format(editor, True) for editor in editors]

    if with_suffix:
        return words[
            join(sep=', ', sep2=', and ', last_sep=', and ')[formatted_names],
            "(Eds.)" if len(editors) > 1 else "(Ed.)"
        ].format_data(context)

    return join(sep=', ', sep2=', and ', last_sep=', and ')[
        formatted_names
    ].format_data(context)

class ACMStyle(BaseStyle):
    name = 'acm'
    default_name_style = 'firstlast'
    default_sorting_style = 'author_year_title'
    #default_label_style = 'acm'

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.abbreviate_names = True

    def format_names(self, role, as_sentence=True):
        formatted_names = acm_names(role)
        if as_sentence:
            return sentence(capfirst=False)[formatted_names]
        else:
            return formatted_names

    def format_editor(self, e, as_sentence=True):
        editors = self.format_names('editor', as_sentence=False)
        if 'editor' not in e.persons:
            # when parsing the template, a FieldIsMissing exception
            # will be thrown anyway; no need to do anything now,
            # just return the template that will throw the exception
            return optional[field('editor')]
        if len(e.persons['editor']) > 1:
            word = '(Eds.)'
        else:
            word = '(Ed.)'
        result = join(sep=' ')[editors, word]
        if as_sentence:
            return sentence[result]
        else:
            return result

    def format_volume(self, e, for_article=False):
        prefix = "Vol."
        if for_article:
            return field('volume')
        else:
            return optional[together[prefix, field('volume')]]

    def format_number(self, e):
        prefix = "No."
        return optional[together[prefix, field('number')]]

    def format_edition(self, e):
        suffix = "ed."
        return optional['(',field('edition'),' ',suffix,')']

    def format_title(self, e, which_field, as_sentence=True):
        formatted_title = field(
            which_field, apply_func=lambda text: text.capitalize()
        )
        if as_sentence:
            return sentence[formatted_title]
        else:
            return formatted_title

    def format_btitle(self, e, which_field, as_sentence=True):
        formatted_title = tag('em')[field(which_field)]
        if as_sentence:
            return sentence[formatted_title]
        else:
            return formatted_title

    def format_web_refs(self, e):
        return sentence(add_period=False)[
            optional[self.format_url(e)],
            optional[self.format_eprint(e)],
            optional[self.format_pubmed(e)],
            optional[self.format_doi(e)],
        ]

    def format_url(self, e):
        return words[
            'URL:',
            href[
                field('url', raw=True),
                field('url', raw=True)
            ]
        ]

    def format_pubmed(self, e):
        return href[
            join[
                'https://www.ncbi.nlm.nih.gov/pubmed/',
                field('pubmed', raw=True)
            ],
            join[
                'PMID:',
                field('pubmed', raw=True)
            ]
        ]

    def format_doi(self, e):
        return href[
            join[
                'https://doi.org/',
                field('doi', raw=True)
            ],
            join[
                'doi:',
                field('doi', raw=True)
            ]
        ]

    def format_eprint(self, e):
        return href[
            join[
                'https://arxiv.org/abs/',
                field('eprint', raw=True)
            ],
            join[
                'arXiv:',
                field('eprint', raw=True)
            ]
        ]

    def format_date(self, e):
        return sentence[
            join["(", first_of[optional[date], "n.d."], ")"]
        ]

    def get_article_template(self, e):
        return toplevel[
            self.format_names('author'),
            sentence[field('year')],
            self.format_title(e, 'title'),
            sentence(sep=', ')[
                words(sep=' ')[
                    tag('em')[field('journal')],
                    optional[self.format_volume(e, for_article=True)],
                ],
                words(sep=' ')[
                    optional[field('number')],
                    join['(',join(sep=' ')[optional_field('month'),field('year')],')'],
                ],
                optional[field('pages')],
            ],
            sentence[optional_field('note')],
            self.format_web_refs(e),
        ]

    def get_book_template(self, e):
        #print(e.fields)
        #print(e.persons)
        return toplevel[
            self.format_names('author'),
            sentence[field('year')],
            sentence(sep=' ')[
                field('title'),
                self.format_edition(e),
            ],
            sentence(sep=', ')[
                    optional_field('series'),
                    self.format_volume(e),
                    optional[self.format_number(e)],
                    self.format_editor(e, as_sentence=False)
            ],
            sentence(sep=', ')[
                field('publisher'),
                optional_field('address')],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]

    def get_booklet_template(self, e):
        if 'month' in e.fields:
            month_year = join['(',field('month'),' ',field('year'),')']
        else: month_year = None
        return toplevel[
            self.format_names('author'),
            sentence[field('year')],
            sentence(sep=' ')[
                field('title'),
                self.format_edition(e),
            ],
            sentence(sep=', ')[
                    optional_field('series'),
                    self.format_volume(e),
                    optional[self.format_number(e)],
                    self.format_editor(e, as_sentence=False)
            ],
            sentence(sep=', ')[
                field('organization'),
                optional[field('publisher')],
                optional_field('address'),
                optional_field('howpublished'),
                month_year,
                optional[field('pages')]],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]

    def get_inbook_template(self, e):
        return toplevel[
            self.format_names('author'),
            sentence[field('year')],
            sentence(sep=' ')[
                field('title'),
                self.format_edition(e),
            ],
            sentence["In",
                    words(sep=', ')[ 
                        optional[self.format_editor(e, as_sentence=False)],
                        self.format_btitle(e, 'booktitle', as_sentence=False),
                        optional_field('type'),
                        optional_field('series'),
                        self.format_volume(e),
                        optional[self.format_number(e)],
                        ],
            ],
            sentence(sep=', ')[
                field('publisher'),
                optional_field('address'),
                optional[field('pages')]],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]
    
    def get_incollection_template(self, e):
        return toplevel[
            self.format_names('author'),
            sentence[field('year')],
            sentence(sep=' ')[
                field('title'),
                self.format_edition(e),
            ],
            sentence["In",
                    words(sep=', ')[ 
                        optional[self.format_editor(e, as_sentence=False)],
                        self.format_btitle(e, 'booktitle', as_sentence=False),
                        optional_field('type'),
                        optional_field('series'),
                        self.format_volume(e),
                        optional[self.format_number(e)],
                        ],
            ],
            sentence(sep=', ')[
                field('publisher'),
                optional_field('address'),
                optional[field('pages')]],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]

    def get_inproceedings_template(self, e):
        if 'month' in e.fields:
            month_year = join['(',field('month'),' ',field('year'),')']
        else: month_year = None
        return toplevel[
            self.format_names('author'),
            sentence[field('year')],
            sentence(sep=' ')[
                field('title'),
                self.format_edition(e),
            ],
            sentence["In",
                    words(sep=', ')[ 
                        optional[self.format_editor(e, as_sentence=False)],
                        self.format_btitle(e, 'booktitle', as_sentence=False),
                        optional_field('type'),
                        optional_field('series'),
                        self.format_volume(e),
                        optional[self.format_number(e)],
                        ],
            ],
            sentence(sep=', ')[
                optional_field('organization'),
                field('publisher'),
                optional_field('address'),
                month_year,
                optional[field('pages')]],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]
        
    def get_manual_template(self, e):
        if 'month' in e.fields:
            month_year = join['(',field('month'),' ',field('year'),')']
        else: month_year = None
        return toplevel[
            optional[self.format_names('author')],
            sentence[field('year')],
            sentence(sep=' ')[
                field('title'),
                self.format_edition(e),
            ],
            sentence(sep=', ')[
                optional_field('organization'),
                optional_field('address'),
                month_year
            ],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]

    def get_misc_template(self, e):
        if 'month' in e.fields:
            month_year = join['(',field('month'),' ',field('year'),')']
        else: month_year = None
        if 'author' in e.persons:
            return toplevel[
                    self.format_names('author'),
                    sentence[field('year')],
                    optional[sentence[self.format_btitle(e, 'title')]],
                    sentence[month_year],
                    sentence[optional_field('note')],
                    self.format_web_refs(e)
            ]
        else:
            return toplevel[
                optional[sentence[self.format_btitle(e, 'title')]],
                sentence[month_year],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]

    def get_phdthesis_template(self, e):
        if 'month' in e.fields:
            month_year = join['(',field('month'),' ',field('year'),')']
        else: month_year = None
        return toplevel[
            self.format_names('author'),
            sentence[field('year')],
            sentence[field('title')],
            'Ph.D. Dissertation.',
            sentence(sep=', ')[
                optional_field('school'),
                optional_field('address'),
                month_year,
            ],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]

    def get_proceedings_template(self, e):
        if 'month' in e.fields:
            month_year = join['(',field('month'),' ',field('year'),')']
        else: month_year = None
        if 'editor' in e.persons:
            return toplevel[
                self.format_editor(e),
                sentence[field('year')],
                sentence(sep=', ')[
                    field('title'),
                    optional_field('series'),
                    self.format_volume(e),
                    optional[self.format_number(e)],
                ],
                sentence(sep=', ')[
                    optional_field('organization'),
                    optional_field('publisher'),
                    optional_field('address'),
                    month_year,
                    optional_field('pages')
                ],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]
        else:
            return toplevel[
                sentence(sep=', ')[
                    field('title'),
                    optional_field('series'),
                    self.format_volume(e),
                    optional[self.format_number(e)],
                ],
                sentence(sep=', ')[
                    optional_field('organization'),
                    optional_field('publisher'),
                    optional_field('address'),
                    month_year,
                    optional_field('pages')
                ],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]

    def get_techreport_template(self, e):
        if 'month' in e.fields:
            month_year = join['(',field('month'),' ',field('year'),')']
        else: month_year = None
        if 'author' in e.persons:
            return toplevel[
                self.format_names('author'),
                sentence[field('year')],
                sentence[field('title')],
                sentence[field('number')],
                sentence(sep=', ')[
                    optional_field('institution'),
                    optional_field('address'),
                    month_year
                ],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]
        else:
            return toplevel[
                sentence[self.format_btitle(e, 'title')],
                sentence[field('number')],
                sentence(sep=', ')[
                    optional_field('institution'),
                    optional_field('address'),
                    month_year
                ],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]

    def get_unpublished_template(self, e):
        if 'month' in e.fields:
            month_year = join['(',field('month'),' ',field('year'),')']
        else: month_year = None
        if 'author' in e.persons:
            return toplevel[
                self.format_names('author'),
                sentence[field('year')],
                sentence[field('title')],
                sentence[field('number')],
                sentence(sep=', ')[
                    optional_field('institution'),
                    optional_field('address'),
                    month_year
                ],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]
        else:
            return toplevel[
                sentence[self.format_btitle(e, 'title')],
                sentence[field('number')],
                sentence(sep=', ')[
                    optional_field('institution'),
                    optional_field('address'),
                    month_year
                ],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]

In [14]:
# Erstellen Sie eine Instanz der ACM-Stilklasse
acm_style = ACMStyle()
plain_backend = find_plugin('pybtex.backends', 'text')()

# Funktion, die Bibtex-Code in einen Literaturstring umwandelt
def umwandler(bibtex_code):
    
        # BibTeX-Code parsen
        bib_data = parse_string(bibtex_code, 'bibtex')

        # Ändern des Eintragstyps in Semantic Scholar von 'inproceedings' zu 'proceedings', von 'masterthesis' zu 'phdthesis',
        # weil dort booktitle fehlt. Allgemein ist in SemScholar die Datenquali sehr schlecht
        # von 'conference' zu 'inproceedings'
        for entry_key in list(bib_data.entries.keys()):
            entry = bib_data.entries[entry_key]
            #if entry.type.lower() == 'inproceedings':
                #entry.type = 'manual'
            if entry.type.lower() == 'conference':
                entry.type = 'inproceedings'
            if entry.type.lower() == 'masterthesis':
                entry.type = 'phdthesis'
            if entry.type == 'InCollection':
                entry.type = 'incollection'
        
        # Einträge formatieren
        formatted_entries = acm_style.format_entries(bib_data.entries.values())
            
        # Custom Backend zum Erfassen der Ausgabe in einem String
        class StringBackend(plain_backend.__class__):
            def __init__(self):
                super().__init__()
                self.output = io.StringIO()
        
            def write_entry(self, label, text):
                self.output.write(text + "\n")
        
            def get_output(self):
                return self.output.getvalue()

            def format_url(self, url):
                return Text(url)
        
        # Custom Backend verwenden
        string_backend = StringBackend()
        try:
            for entry in formatted_entries:
                string_backend.write_entry(entry.key, entry.text.render(string_backend))
        except FieldIsMissing as ex:
            return ex

        # Formatierten String erhalten
        formatted_string = string_backend.get_output()
       
        # Nachbearbeitung: \url{}-Kommando entfernen
        formatted_string = formatted_string.replace('\\url ', '')
    
        return formatted_string

# Funktion, die den Literaturtyp aus dem Bibtex-Code ausliest
def entrytype(bibtex_code):
    # BibTeX-Code parsen
    bib_data = parse_string(bibtex_code, 'bibtex')

    #Extrahiere Eintragstyp
    entry = list(bib_data.entries.values())[0]
    return entry.type

In [15]:
#Erzeugung der Spalte mit den Literaturtypen
df["Literaturtyp"] = df["BibTeX"].apply(entrytype)
df5["Literaturtyp"] = df5["BibTeX"].apply(entrytype)
df6["Literaturtyp"] = df6["BibTeX"].apply(entrytype)
df7["Literaturtyp"] = df7["BibTeX"].apply(entrytype)
#display(df)

In [16]:
#Filtern nach einem Literaturtyp und Beschränkung der Einträge auf 700 für Trainingsdaten und 300 für die Testdaten
df2 = df[df["Literaturtyp"] == 'article']
#df2 = df2.head(700)
df2 = df2.iloc[700:1000]

df3 = df[df["Literaturtyp"] == 'proceedings']
#df3 = df3.head(700)
df3 = df3.iloc[700:1000]

df4 = df[df["Literaturtyp"] == 'inproceedings']
#df4 = df4.head(700)
df4 = df4.iloc[700:1000]

#df5 = df5.head(700)
df5 = df5.iloc[700:1000]
#df6 = df6.head(700)
df6 = df6.iloc[700:1000]
#df7 = df7.head(700)
df7 = df7.iloc[700:1000]

#Vereinigung zu einem Dataframe
df_acm = pd.concat([df2,df3,df4,df5,df6,df7], ignore_index=True)

display(df_acm.head(5))
display(df_acm.info())

,BibTeX,Literaturtyp
0,"@article{mansouri-bigvand-etal-2017-joint,\n ...",article
1,"@article{zhang-etal-2017-aspect,\n title = ...",article
2,"@article{gallagher-etal-2017-anchored,\n ti...",article
3,"@article{joty-etal-2017-discourse,\n title ...",article
4,"@article{rozovskaya-etal-2017-adapting,\n t...",article


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   BibTeX        1800 non-null   object
 1   Literaturtyp  1800 non-null   object
dtypes: object(2)
memory usage: 28.3+ KB


None

In [17]:
#Erzeugung der Spalte mit den Literaturstrings
df_acm["Referenzstring"] = df_acm["BibTeX"].apply(umwandler)

display(df_acm.head(5))

,BibTeX,Literaturtyp,Referenzstring
0,"@article{mansouri-bigvand-etal-2017-joint,\n ...",article,"Anahita Mansouri Bigvand, Te Bu, and Anoop Sar..."
1,"@article{zhang-etal-2017-aspect,\n title = ...",article,"Yuan Zhang, Regina Barzilay, and Tommi Jaakkol..."
2,"@article{gallagher-etal-2017-anchored,\n ti...",article,"Ryan J. Gallagher, Kyle Reing, David Kale, and..."
3,"@article{joty-etal-2017-discourse,\n title ...",article,"Shafiq Joty, Francisco Guzmán, Lluís Màrquez, ..."
4,"@article{rozovskaya-etal-2017-adapting,\n t...",article,"Alla Rozovskaya, Dan Roth, and Mark Sammons. 2..."


In [18]:
ind = 500
print(df_acm["Referenzstring"][ind])
print(df_acm["BibTeX"][ind])
print(df_acm["Literaturtyp"][ind])

Malvina Nissim, Viviana Patti, Barbara Plank, and Claudia Wagner (Eds.). 2018. Proceedings of the Second Workshop on Computational Modeling of People's Opinions, Personality, and Emotions in Social Media. Association for Computational Linguistics, New Orleans, Louisiana, USA, (June 2018). URL: https://aclanthology.org/W18-1100, doi:10.18653/v1/W18-11

@proceedings{ws-2018-modeling,
    title = "Proceedings of the Second Workshop on Computational Modeling of People{'}s Opinions, Personality, and Emotions in Social Media",
    editor = "Nissim, Malvina  and
      Patti, Viviana  and
      Plank, Barbara  and
      Wagner, Claudia",
    month = jun,
    year = "2018",
    address = "New Orleans, Louisiana, USA",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/W18-1100",
    doi = "10.18653/v1/W18-11",
}

proceedings


In [19]:
# Suche nach Strings, die mit "missing" beginnen; aus irgendeinem Grund muss der Referenzstring als String behandelt werden
result = df_acm[df_acm['Referenzstring'].astype(str).str.startswith('missing')]

# Ergebnis anzeigen
display(result)

,BibTeX,Literaturtyp,Referenzstring
10,"@article{nn-2017-reviewers,\n title = ""Revi...",article,missing author in nn-2017-reviewers
99,"@article{nn-2016-reviewers,\n title = ""Revi...",article,missing author in nn-2016-reviewers
284,"@article{nn-2014-erratum,\n title = ""Erratu...",article,missing author in nn-2014-erratum
984,"@incollection{pGRA91a,\n author = ""W. K. Gra...",incollection,missing publisher in pGRA91a
993,"@incollection {vOWE03b,\n author = \t {A. B. ...",incollection,missing publisher in vOWE03b
1007,"@INCOLLECTION{pKUR01a,\n author={D. Kurowicka ...",incollection,missing publisher in pKUR01a
1017,"@INCOLLECTION {rDUD95b,\n\tAUTHOR=""L. T. Bernh...",incollection,missing publisher in rDUD95b
1026,"@incollection {rHEL98bx,\n author = {Helleka...",incollection,missing year in rHEL98bx
1029,"@incollection{rHOW07a,\n author = {L. How...",incollection,missing booktitle in rHOW07a
1031,"@incollection {rLAR98ax,\n AUTHOR =""G. La...",incollection,missing year in rLAR98ax


In [20]:
# Filtern und Zeilen löschen, die mit "missing" beginnen
df_acm = df_acm[~df_acm['Referenzstring'].astype(str).str.startswith('missing')]

# Suche nach Strings, die mit "missing" beginnen; aus irgendeinem Grund muss der Referenzstring als String behandelt werden
result = df_acm[df_acm['Referenzstring'].astype(str).str.startswith('missing')]

# Ergebnis anzeigen
display(result)

,BibTeX,Literaturtyp,Referenzstring


In [21]:
# Neue Spalte 'Style' einfügen nach der ersten Spalte (Index 1)
df_acm.insert(1, 'Style', 'acm')

# Neuordnung der Spalten
new_order = ['Referenzstring', 'Style', 'Literaturtyp', 'BibTeX']
df_acm = df_acm[new_order]
display(df_acm)

# DataFrame als CSV-Datei mit Pipe-Separator exportieren
#df_acm.to_csv('training_acm.csv', sep='|', index=False, encoding='utf-8')
df_acm.to_csv('test_acm.csv', sep='|', index=False, encoding='utf-8')

,Referenzstring,Style,Literaturtyp,BibTeX
0,"Anahita Mansouri Bigvand, Te Bu, and Anoop Sar...",acm,article,"@article{mansouri-bigvand-etal-2017-joint,\n ..."
1,"Yuan Zhang, Regina Barzilay, and Tommi Jaakkol...",acm,article,"@article{zhang-etal-2017-aspect,\n title = ..."
2,"Ryan J. Gallagher, Kyle Reing, David Kale, and...",acm,article,"@article{gallagher-etal-2017-anchored,\n ti..."
3,"Shafiq Joty, Francisco Guzmán, Lluís Màrquez, ...",acm,article,"@article{joty-etal-2017-discourse,\n title ..."
4,"Alla Rozovskaya, Dan Roth, and Mark Sammons. 2...",acm,article,"@article{rozovskaya-etal-2017-adapting,\n t..."
...,...,...,...,...
1795,Mélodie Greenlees. 2012. Intelligente technisc...,acm,phdthesis,"@phdthesis{Greenlees.2012,\n author = {Greenle..."
1796,Personnalisée Greenly. 2013. Database techniqu...,acm,phdthesis,"@phdthesis{Greenly.2013,\n author = {Greenly, ..."
1797,Garçon Greeve. 2018. Anforderungsbasierte Mode...,acm,phdthesis,"@phdthesis{Greeve.2018,\n author = {Greeve, Ga..."
1798,Ruò Glasscoo. 2014. Temporal models of streami...,acm,phdthesis,"@phdthesis{Glasscoo.2014,\n author = {Glasscoo..."
